<a href="https://colab.research.google.com/github/takuxp/bigdata_study/blob/main/%EA%B8%B0%EC%B6%9C%EC%9C%A0%ED%98%95_4%ED%9A%8C(%EC%9E%91%EC%97%85%ED%98%952).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Q. [마케팅] 자동차 시장 세분화
- 자동차 회사는 새로운 전략을 수립하기 위해 4개의 시장으로 세분화했습니다.
- 기존 고객 분류 자료를 바탕으로 신규 고객이 어떤 분류에 속할지 예측해주세요!


- 예측할 값(y): "Segmentation" (1,2,3,4)
- 평가: Macro f1-score
- data: train.csv, test.csv
- 제출 형식:
~~~
ID,Segmentation
458989,1
458994,2
459000,3
459003,4
~~~

### 답안 제출 참고
- 아래 코드 예측변수와 수험번호를 개인별로 변경하여 활용
- pd.DataFrame({'ID': test.ID, 'Segmentation': pred}).to_csv('003000000.csv', index=False)

### 노트북 구분
- basic: 수치형 데이터만 활용 -> 학습 및 test데이터 예측
- intermediate: 범주형 데이터도 활용 -> 학습 및 test데이터 예측
- advanced: 학습 및 교차 검증(모델 평가) -> 하이퍼파라미터 튜닝 -> test데이터 예측

In [81]:
# 라이브러리 불러오기
import pandas as pd

In [82]:
# 데이터 불러오기
train = pd.read_csv("./4회기출작업형2/train.csv")
test = pd.read_csv("./4회기출작업형2/test.csv")

# 🍭 basic 단계 🍭  
- 목표: 수치형 데이터만이라도 활용해 제출하자!!!👍

## EDA

In [83]:
# 데이터 크기 확인
print(train.shape)
print(test.shape)

(6665, 11)
(2154, 10)


In [84]:
# train 샘플 확인
print(train.head())

       ID  Gender Ever_Married  Age Graduated  Profession  Work_Experience  \
0  462809    Male           No   22        No  Healthcare              1.0   
1  466315  Female          Yes   67       Yes    Engineer              1.0   
2  461735    Male          Yes   67       Yes      Lawyer              0.0   
3  461319    Male          Yes   56        No      Artist              0.0   
4  460156    Male           No   32       Yes  Healthcare              1.0   

  Spending_Score  Family_Size  Var_1  Segmentation  
0            Low          4.0  Cat_4             4  
1            Low          1.0  Cat_6             2  
2           High          2.0  Cat_6             2  
3        Average          2.0  Cat_6             3  
4            Low          3.0  Cat_6             3  


In [85]:
# test 샘플 확인
print(test.head())

       ID  Gender Ever_Married  Age Graduated  Profession  Work_Experience  \
0  458989  Female          Yes   36       Yes    Engineer              0.0   
1  458994    Male          Yes   37       Yes  Healthcare              8.0   
2  459000    Male          Yes   59        No   Executive             11.0   
3  459003    Male          Yes   47       Yes      Doctor              0.0   
4  459005    Male          Yes   61       Yes      Doctor              5.0   

  Spending_Score  Family_Size  Var_1  
0            Low          1.0  Cat_6  
1        Average          4.0  Cat_6  
2           High          2.0  Cat_6  
3           High          5.0  Cat_4  
4            Low          3.0  Cat_6  


In [86]:
# target 확인
train.Segmentation.head()

0    4
1    2
2    2
3    3
4    3
Name: Segmentation, dtype: int64

In [87]:
# 결측치 확인(train)
train.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
Segmentation       0
dtype: int64

In [88]:
# 결측치 확인(test)
test.isnull().sum()

ID                 0
Gender             0
Ever_Married       0
Age                0
Graduated          0
Profession         0
Work_Experience    0
Spending_Score     0
Family_Size        0
Var_1              0
dtype: int64

In [89]:
# type 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


## 전처리

In [90]:
# target(y, label) 값 복사
train_target = train.pop('Segmentation')
train_id = train.pop('ID')
print(train_target, train_id)

0       4
1       2
2       2
3       3
4       3
       ..
6660    2
6661    4
6662    4
6663    2
6664    2
Name: Segmentation, Length: 6665, dtype: int64 0       462809
1       466315
2       461735
3       461319
4       460156
         ...  
6660    463002
6661    464685
6662    465406
6663    467299
6664    461879
Name: ID, Length: 6665, dtype: int64


In [91]:
# test데이터 ID 복사
test_id = test.pop('ID')

In [92]:
# 수치형 컬럼(train)
cols = ['Age', 'Work_Experience', 'Family_Size']

In [93]:
# 수치형 컬럼(test)
test.info()
test_cols = ['Age', 'Work_Experience', 'Family_Size']

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2154 entries, 0 to 2153
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Gender           2154 non-null   object 
 1   Ever_Married     2154 non-null   object 
 2   Age              2154 non-null   int64  
 3   Graduated        2154 non-null   object 
 4   Profession       2154 non-null   object 
 5   Work_Experience  2154 non-null   float64
 6   Spending_Score   2154 non-null   object 
 7   Family_Size      2154 non-null   float64
 8   Var_1            2154 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 151.6+ KB


## model 학습 및 예측

In [94]:
# 모델 선택 및 학습
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier()
model = rf.fit(train[cols], train_target)
pred = model.predict(test[cols])
pred

array([2, 3, 3, ..., 4, 3, 1])

In [95]:
# 예측 결과 -> 데이터 프레임
result = pd.DataFrame({
    'ID': test_id,
    'Segmentation': pred
})

In [96]:
result.to_csv('000000.csv', index=False)

# 🍭 intermediate 단계 🍭
- 목표: 범주형(카테고리)데이터 활용하기

In [97]:
# 라이브러리 불러오기
import pandas as pd

In [98]:
# 데이터 불러오기
train = pd.read_csv("./4회기출작업형2/train.csv")
test = pd.read_csv("./4회기출작업형2/test.csv")

## EDA

In [99]:
# train 샘플 확인
train.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,4
1,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,2
2,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,2
3,461319,Male,Yes,56,No,Artist,0.0,Average,2.0,Cat_6,3
4,460156,Male,No,32,Yes,Healthcare,1.0,Low,3.0,Cat_6,3


In [100]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               6665 non-null   int64  
 1   Gender           6665 non-null   object 
 2   Ever_Married     6665 non-null   object 
 3   Age              6665 non-null   int64  
 4   Graduated        6665 non-null   object 
 5   Profession       6665 non-null   object 
 6   Work_Experience  6665 non-null   float64
 7   Spending_Score   6665 non-null   object 
 8   Family_Size      6665 non-null   float64
 9   Var_1            6665 non-null   object 
 10  Segmentation     6665 non-null   int64  
dtypes: float64(2), int64(3), object(6)
memory usage: 572.9+ KB


In [101]:
train.describe(include="O")

,Gender,Ever_Married,Graduated,Profession,Spending_Score,Var_1
count,6665,6665,6665,6665,6665,6665
unique,2,2,2,9,3,7
top,Male,Yes,Yes,Artist,Low,Cat_6
freq,3677,3944,4249,2192,3999,4476


## 전처리

In [102]:
# 원핫 인코딩
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [103]:
# type 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6665 entries, 0 to 6664
Data columns (total 30 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        6665 non-null   int64  
 1   Age                       6665 non-null   int64  
 2   Work_Experience           6665 non-null   float64
 3   Family_Size               6665 non-null   float64
 4   Segmentation              6665 non-null   int64  
 5   Gender_Female             6665 non-null   bool   
 6   Gender_Male               6665 non-null   bool   
 7   Ever_Married_No           6665 non-null   bool   
 8   Ever_Married_Yes          6665 non-null   bool   
 9   Graduated_No              6665 non-null   bool   
 10  Graduated_Yes             6665 non-null   bool   
 11  Profession_Artist         6665 non-null   bool   
 12  Profession_Doctor         6665 non-null   bool   
 13  Profession_Engineer       6665 non-null   bool   
 14  Professi

In [104]:
# target(y, label) 값 복사
target = train.pop('Segmentation')
target

0       4
1       2
2       2
3       3
4       3
       ..
6660    2
6661    4
6662    4
6663    2
6664    2
Name: Segmentation, Length: 6665, dtype: int64

In [105]:
train_id = train.pop('ID')


In [106]:
# test데이터 ID 복사
test_id = test.pop('ID')

## model 학습 및 예측

In [107]:
# 모델 선택 및 학습
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=2024)
model = rf.fit(train, target)
pred = model.predict(test)
pred

array([2, 3, 3, ..., 2, 3, 4])

In [109]:
# 예측 결과 -> 데이터 프레임
submit = pd.DataFrame({
    'ID': test_id,
    'Segmentation': pred
})

In [110]:
submit.to_csv("000000-1.csv", index=False)

# 🍭 advanced 단계 🍭
- 목표: 교차검증 및 평가 후 제출하기

In [ ]:
# 데이터 불러오기
train = pd.read_csv("./4회기출작업형2/train.csv")
test = pd.read_csv("./4회기출작업형2/test.csv")

In [ ]:
# 범주형 변수

In [ ]:
## label encoding

In [ ]:
## cat.codes의 label 인코딩은 ABC 순대로 되는 것을 확인할 수 있다

In [ ]:
## label encoding

In [ ]:
# ID, target 처리

In [ ]:
# 모델 선택
# 하이퍼파라미터 튜닝: max_depth, n_estimators

In [ ]:
# 모델 선택
# 하이퍼파라미터 튜닝: max_depth, n_estimators

In [ ]:
# 학습

In [ ]:
# 예측 결과 -> 데이터 프레임